In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle
import scipy
from sklearn.linear_model import LinearRegression

# from src.sharpe_simulation import * 

In [2]:
[P,F] = pickle.load( open("./data/save.p", "rb") )

In [3]:
P.keys()

dict_keys(['ME10', 'FF25', 'IND10', 'IND30', 'IND49'])

In [4]:
F.keys()

dict_keys(['CAPM', 'FF3'])

In [5]:
P['ME10']

,Lo 10,Dec-02,Dec-03,Dec-04,Dec-05,Dec-06,Dec-07,Dec-08,Dec-09,Hi 10
1959-07-31,0.0363,0.0179,0.0302,0.0332,0.0201,0.0383,0.0239,0.0214,0.0328,0.0333
1959-08-31,-0.0183,-0.0015,-0.0169,-0.0217,-0.0218,-0.0037,-0.0185,-0.0217,-0.0213,-0.0112
1959-09-30,-0.0404,-0.0512,-0.0456,-0.0445,-0.0505,-0.0437,-0.0557,-0.0492,-0.0539,-0.0464
1959-10-31,-0.0040,0.0229,0.0145,0.0261,0.0216,0.0261,0.0109,0.0391,0.0225,0.0066
1959-11-30,0.0000,0.0040,0.0059,0.0285,0.0139,0.0055,0.0181,0.0269,0.0104,0.0161
...,...,...,...,...,...,...,...,...,...,...
2003-08-31,0.0512,0.0486,0.0480,0.0463,0.0559,0.0507,0.0515,0.0473,0.0375,0.0122
2003-09-30,0.0442,0.0004,-0.0242,-0.0282,-0.0264,-0.0151,-0.0144,-0.0180,-0.0177,-0.0106
2003-10-31,0.0939,0.0892,0.0893,0.0861,0.0836,0.0838,0.0757,0.0848,0.0674,0.0506
2003-11-30,0.0486,0.0397,0.0340,0.0330,0.0304,0.0389,0.0315,0.0244,0.0209,0.0059


# CAPM model sample for ME10

In [6]:
T = P['ME10'].shape[0]
N = P['ME10'].shape[1]
L = F['CAPM'].shape[1]
print(T,N,L)

534 10 1


In [7]:
capm_me10 = LinearRegression().fit(F['CAPM'],P['ME10'])
res_me10 = P['ME10'] - capm_me10.predict(F['CAPM'])
alpha_me10 = capm_me10.intercept_
beta_me10 = capm_me10.coef_

In [9]:
unbiased_rescov_me10 = res_me10.T@res_me10/(T-L-1)
unbiased_rescov_me10.shape

(10, 10)

In [10]:
mu = np.mean(F['CAPM'])
unbiased_faccov_me10 = (F['CAPM'] - mu).T@(F['CAPM']- mu)/(T-1)

In [11]:
GRS_statis = (T/N)*(T-N-L)/(T-L-1)*(alpha_me10.reshape(N)@np.linalg.inv(unbiased_rescov_me10)@alpha_me10)/(1+mu.T@unbiased_faccov_me10@mu)
GRS_statis

2.0435755525760753

# function below

In [12]:
def CAPM_GRS(CAPM,ret):
    T = ret.shape[0]
    N = ret.shape[1]
    L = CAPM.shape[1]
    
#     capm_ = linear_model
    capm_ = LinearRegression().fit(CAPM,ret)
    res_ = ret - capm_.predict(CAPM)
    alpha_ = capm_.intercept_
    beta_ = capm_.coef_
    
    unbiased_rescov_ = res_.T@res_/(T-L-1)
    
    mu = np.mean(CAPM)
    unbiased_faccov_ = (CAPM - mu).T@(CAPM- mu)/(T-1)
    
    GRS_statis = (T/N)*(T-N-L)/(T-L-1)*(alpha_.reshape(N)@scipy.linalg.inv(unbiased_rescov_)@alpha_)/(1+mu.T@np.linalg.inv(unbiased_faccov_)@mu)
    
    
    return(GRS_statis)



In [13]:
CAPM_GRS(F['CAPM'],P['FF25'])

3.545059614507397

In [14]:
def Multivariate_GRS(FAC,ret):
    T = ret.shape[0]
    N = ret.shape[1]
    L = FAC.shape[1]
    
#     capm_ = linear_model
    fac_ = LinearRegression().fit(FAC,ret)
    res_ = ret - fac_.predict(FAC)
    alpha_ = fac_.intercept_
    beta_ = fac_.coef_
    
    unbiased_rescov_ = res_.T@res_/(T-L-1)
    
    mu = np.mean(FAC)
    unbiased_faccov_ = (FAC - mu).T@(FAC- mu)/(T-1)
    
    GRS_statis = (T/N)*(T-N-L)/(T-L-1)*(alpha_.T@scipy.linalg.inv(unbiased_rescov_)@alpha_)/(1+mu.T@np.linalg.inv(unbiased_faccov_)@mu)
    
    
    return(GRS_statis)



In [15]:
Multivariate_GRS(F['FF3'],P['IND10'])

3.5794017491224146

In [16]:
GRS_statistics = pd.DataFrame(columns = P.keys(), index = F.keys())
for i in F.keys():
    for j in P.keys():
        if i == 'CAPM':
            GRS_statistics.loc[i,j] = CAPM_GRS(F[i],P[j])
        else:
            GRS_statistics.loc[i,j] = Multivariate_GRS(F[i],P[j])


,ME10,FF25,IND10,IND30,IND49
CAPM,2.021477,3.54506,1.843207,1.243852,4.492476
FF3,1.475354,2.381649,3.579402,2.259423,5.220047
